In [2]:
# Imports
import torch
import torch.nn as nn
import sys
import pandas as pd

In [ ]:
# Dataset preparations -- GitHub

In [3]:
# Model
# Model assumes that there is at least 1 dimension in monthly, yearly, and static
class YieldLSTMMLPConnected(nn.Module):
    def __init__(self,
                    monthly_dim=7,     # Avg's by month (seq features, should be 7 features from prism data)
                    monthly_hidden=64,
                    monthly_layers=1,
                    yearly_dim=5,    # number of yearly features (should be 5 features, 4 from nasa + the annual yield by county)
                    static_dim=8,    # number of static features (should be 8 features from soil data)
                    yearly_hidden=32,
                    static_hidden=32,
                    head_hidden=64,
                    output_dim=1,
                    dropout=0.1
                    ):
        super().__init__()

        # Monthly branch LSTM
        self.lstm = nn.LSTM(
            input_size=monthly_dim,
            hidden_size=monthly_hidden,
            num_layers=monthly_layers,
            batch_first=True,
            bidirectional=False
        )
        self.monthly_proj = nn.Sequential(
            nn.Linear(monthly_hidden, monthly_hidden),
            nn.ReLU(),
            nn.Dropout(dropout)
        )

        # Yearly branch MLP
        self.yearly_proj = nn.Sequential(
            nn.Linear(yearly_dim, yearly_hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(yearly_hidden, yearly_hidden),
            nn.ReLU(),
        )

        # Static branch MLP
        self.static_proj = nn.Sequential(
            nn.Linear(static_dim, static_hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(static_hidden, static_hidden),
            nn.ReLU(),
        )

        # Combined head (output of combined branches)
        combined_dim = monthly_hidden + yearly_hidden + static_hidden
        self.head = nn.Sequential(
            nn.Linear(combined_dim, head_hidden),
            nn.ReLU(),
            nn.Droput(dropout),
            nn.Linear(head_hidden, head_hidden//2),
            nn.ReLU(),
            nn.Linear(head_hidden//2, output_dim)
        )

    def forward(self, monthly, yearly, static):
        feats = []

        # Monthly shape (batch, seq_len, monthly_dim)
        # LSTM: take last hidden state
        lstm_out, (h_n, c_n) = self.lstm(monthly)
        # h_n shape: (num_layers, batch, hidden)
        last_h = h_n[-1] # (batch, monthly_hidden)
        monthly_emb = self.monthly_proj(last_h)
        feats.append(monthly_emb)

        feats.append(self.yearly_proj(yearly))

        feats.append(self.static_proj(static))

        combined = torch.cat(feats, dim=1)
        out = self.head(combined)
        return out

In [ ]:
# Training